In [1]:
!pwd
%ls
%cd /
%ls

/mnt/workspace
glm3ft.ipynb  ZhipuAI-chatglm3-6b.ipynb
/
'=0.21.0'                     dev/     libx32@                    proc/   tmp/
'=0.6.0'                      etc/     media/                     root/   usr/
'=2.25.0'                     home/    mnt/                       run/    var/
 bin@                         lib@     modelscope/                sbin@
 boot/                        lib32@   NGC-DL-CONTAINER-LICENSE   srv/
 cuda-keyring_1.0-1_all.deb   lib64@   opt/                       sys/


In [1]:
!nvidia-smi

Fri May  3 15:28:47 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:00:07.0 Off |                    0 |
|  0%   33C    P8    22W / 150W |      0MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorrt as tr
import tensorflow as tf
import torch

print(f"tf cuda available：{tf.config.list_physical_devices('GPU')}")

# 输出带CPU，表示torch是CPU版本的，否则会是+cuxxx
print(f'torch的版本是：{torch.__version__}')
print(f'torch是否能使用cuda：{torch.cuda.is_available()}')

In [11]:
pip install 'accelerate>=0.27.2' modelscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 575.6 kB/s eta 0:00:0000:0100:05
Note: you may need to restart the kernel to use updated packages.


# 下载模型


In [3]:
from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", cache_dir='chatglm3-6b', revision = "master")


2024-05-06 14:37:39,526 - modelscope - INFO - PyTorch version 2.2.1+cu121 Found.
2024-05-06 14:37:39,529 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-05-06 14:37:39,530 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-06 14:37:39,638 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 de5c0a4eb686197c6f2084aeb7788046 and a total number of 976 components indexed
2024-05-06 14:37:39,902 - modelscope - WARNING - Using the master branch is fragile, please use it with caution!
2024-05-06 14:37:39,903 - modelscope - INFO - Use user-specified model revision: master
Downloading: 100%|██████████| 1.29k/1.29k [00:00<00:00, 4.53MB/s]
Downloading: 100%|██████████| 37.0/37.0 [00:00<00:00, 101kB/s]
Downloading: 100%|██████████| 2.28k/2.28k [00:00<00:00, 5.77MB/s]
Downloading: 100%|█████████▉| 1.70G/1.70G [00:23<00:00, 76.7MB/s]
Downloading: 100%|

In [5]:
!git config --global http.postBuffer 524288000 && git clone https://github.com/hiyouga/LLaMA-Factory.git
!git clone https://github.com/kangtsang/ft_data.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 10765, done.
remote: Counting objects: 100% (1581/1581), done.
remote: Compressing objects: 100% (326/326), done.
remote: Total 10765 (delta 1316), reused 1431 (delta 1249), pack-reused 9184
Receiving objects: 100% (10765/10765), 214.50 MiB | 13.89 MiB/s, done.
Resolving deltas: 100% (7912/7912), done.
Updating files: 100% (209/209), done.


In [6]:
!git clone https://github.com/kangtsang/ft_data.git

Cloning into 'ft_data'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 245 (delta 2), reused 13 (delta 1), pack-reused 231
Receiving objects: 100% (245/245), 59.95 MiB | 10.47 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [8]:
!cd LLaMA-Factory && pwd && git checkout -b 'master'

/mnt/workspace/LLaMA-Factory
切换到一个新分支 'master'


In [7]:
!cd LLaMA-Factory && pwd && pip install -r ./requirements.txt

/hy-tmp/LLaMA-Factory
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 958.4 kB/s eta 0:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 303.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 391.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 852.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 774.3 kB/s eta 0:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 449.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 108.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 528.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 157.2 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 946.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━

In [ ]:
!cd LLaMA-Factory && pwd && pip install -e .[metrics]

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
Processing ./LLaMA-Factory
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 92.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 36.0 MB/s eta 0:00:00


In [13]:
!CUDA_VISIBLE_DEVICES=0 python LLaMA-Factory/src/train.py \
    --stage sft \
    --do_train True \
    --model_name_or_path /hy-tmp/chatglm3-6b/ZhipuAI/chatglm3-6b \
    --quantization_bit 4 \
    --finetuning_type lora \
    --template chatglm3 \
    --dataset_dir ft_data \
    --dataset classify \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 35 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --report_to none \
    --output_dir saves/glm3_train_2024-05-06 \
    --fp16 True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --lora_target query_key_value \
    --plot_loss True 

05/06/2024 14:59:54 - WARNING - llmtuner.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
05/06/2024 14:59:54 - INFO - llmtuner.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2085] 2024-05-06 14:59:54,686 >> loading file tokenizer.model
[INFO|tokenization_utils_base.py:2085] 2024-05-06 14:59:54,686 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2085] 2024-05-06 14:59:54,686 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2085] 2024-05-06 14:59:54,686 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2085] 2024-05-06 14:59:54,686 >> loading file tokenizer.json
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
05/06/2024 14:59:55 - INFO - llmtuner.data.template - Add <|use

In [15]:
import torch

torch.cuda.empty_cache()

print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())


0
0


In [7]:
!pwd
# /mnt/workspace

/mnt/workspace


In [ ]:
# 模型对话 /mnt/workspace/chatglm3-6b

from llmtuner import ChatModel
from llmtuner.extras.misc import torch_gc
chat_model = ChatModel(dict(
  model_name_or_path="/mnt/workspace/chatglm3-6b/ZhipuAI/chatglm3-6b",     # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="saves/glm3_train_2024-05-03",   # 加载之前保存的 LoRA 适配器
  finetuning_type="lora",                  # 和训练保持一致
  template="chatglm3",                     # 和训练保持一致
  quantization_bit=8,                      # 加载 4 比特量化模型
))
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break

  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})     # 把提示词添加到消息中
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):      # 流式输出
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response}) # 把回答添加到消息中

torch_gc()

In [ ]:
from modelscope import AutoTokenizer, AutoModel, snapshot_download
model_dir = snapshot_download("chatglm3-6b", revision = "v1.0.0")
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModel.from_pretrained(model_dir, trust_remote_code=True).half().cuda()
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)